In [ ]:
from openai import OpenAI
import os 
os.environ['OPENAI_API_KEY'] = ''
client = OpenAI(
  organization='org-T4ew5eiTxSpJrQVcBLLZQhTF',
  project='proj_GJ4tiZkw4JfcR6nEhgZlDI39',
)



In [22]:
import json
file = 'patch_api\\cross\\cross_project_patch_batch_0.jsonl'
with open(file) as ff:
    for line in ff.readlines():
        obj = json.loads(line.strip())
        body = obj['body']
        print(body)
        chat_completion = client.chat.completions.create(model=body['model'],messages=body['messages'],max_tokens=body['max_tokens'],temperature=body['temperature'])
        break
chat_completion

{'model': 'gpt-3.5-turbo', 'messages': [{'role': 'user', 'content': 'I want you to act as a commit message generator.\nDo not write any explanations or other words, just reply with the commit message.\nPlease generate the message of this changes:\nmmm usr.sbin/smtpd/table.c <s> ppp usr.sbin/smtpd/table.c <s> -/* $OpenBSD: table.c,v 1.48 2019/01/10 07:40:52 eric Exp $ */ \n +/* $OpenBSD: table.c,v 1.49 2020/12/23 08:12:14 martijn Exp $ */ \n  \n /* \n * Copyright (c) 2013 Eric Faurot <eric@openbsd.org> \n table_regex_match(const char *string, const char *pattern) \n { \n regex_t preg; \n int cflags = REG_EXTENDED|REG_NOSUB; \n + int ret; \n  \n if (strncmp(pattern, "(?i)", 4) == 0) { \n cflags |= REG_ICASE; \n table_regex_match(const char *string, const char *pattern) \n if (regcomp(&preg, pattern, cflags) != 0) \n return (0); \n  \n - if (regexec(&preg, string, 0, NULL, 0) != 0) \n + ret = regexec(&preg, string, 0, NULL, 0); \n + \n + regfree(&preg); \n + \n + if (ret != 0) \n return (

ChatCompletion(id='chatcmpl-9MUIMqLTeafnsqCgrrvzIuunAWdaX', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='table: refactor regex matching logic', role='assistant', function_call=None, tool_calls=None))], created=1715146698, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=6, prompt_tokens=325, total_tokens=331))

In [25]:
chat_completion.choices[0].message.content

'table: refactor regex matching logic'

In [16]:
response = client.files.list()
response

SyncPage[FileObject](data=[FileObject(id='file-vCZFbZuWP2BoEM8mRPeRz1HL', bytes=586819, created_at=1715144345, filename='cross_project_patch_batch_2.jsonl', object='file', purpose='batch', status='processed', status_details=None), FileObject(id='file-pE6ExqmWCkBNSOHi7L6ORGfP', bytes=447704, created_at=1715144282, filename='cross_project_patch_batch_10.jsonl', object='file', purpose='batch', status='processed', status_details=None), FileObject(id='file-yx1Bp6jn0bHlXZpurasOqcsb', bytes=590078, created_at=1715144220, filename='cross_project_patch_batch_1.jsonl', object='file', purpose='batch', status='processed', status_details=None), FileObject(id='file-DcDfmTr3lOShgsmzymW8ggZk', bytes=590674, created_at=1715144150, filename='cross_project_patch_batch_0.jsonl', object='file', purpose='batch', status='processed', status_details=None)], object='list', has_more=False)

In [17]:
for f in response:
    print(f.id)
    client.files.delete(f.id)

file-vCZFbZuWP2BoEM8mRPeRz1HL
file-pE6ExqmWCkBNSOHi7L6ORGfP
file-yx1Bp6jn0bHlXZpurasOqcsb
file-DcDfmTr3lOShgsmzymW8ggZk


In [3]:
import glob 
import json
map_key_gold = dict()
map_key_pred = dict()
c = 0
for file in glob.glob('patch_api/cross/output_*'):
    with open(file) as f:
        for l in f.readlines():
            obj = json.loads(l.strip())
            msg = obj['response']['body']['choices'][0]['message']['content']
            msg = ' '.join(msg.split())
            c += 1
            map_key_pred[obj['custom_id']] = str(c) + "\t"+ msg
            map_key_pred[obj['custom_id']] = msg
        

with open('patch_api/cross/test_cross_project_patch.jsonl') as f:
    for l in f.readlines():
        data  = json.loads(l.strip())
        map_key_gold[data['index']] = data['label']

print(c,len(map_key_gold),len(map_key_pred))

assert len(map_key_pred) == len(map_key_pred)

5952 5952 5952


In [4]:
golds,preds = list(),list()
for k, v in map_key_pred.items():
    preds.append(v.strip())
    golds.append(map_key_gold[k].strip())
with open('patch_api/cross/gold','w+') as f:
    f.write('\n'.join(golds))
with open('patch_api/cross/prd','w+') as f:
    f.write('\n'.join(preds))